In [1]:
import sys
import os
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '../..')))

# Path Configuration
from tools.preprocess import *

# Processing context
trait = "Metabolic_Rate"
cohort = "GSE89231"

# Input paths
in_trait_dir = "../../input/GEO/Metabolic_Rate"
in_cohort_dir = "../../input/GEO/Metabolic_Rate/GSE89231"

# Output paths
out_data_file = "../../output/preprocess/Metabolic_Rate/GSE89231.csv"
out_gene_data_file = "../../output/preprocess/Metabolic_Rate/gene_data/GSE89231.csv"
out_clinical_data_file = "../../output/preprocess/Metabolic_Rate/clinical_data/GSE89231.csv"
json_path = "../../output/preprocess/Metabolic_Rate/cohort_info.json"


### Step 1: Initial Data Loading

In [2]:
# 1. Check what files are actually in the directory
import os
print("Files in the directory:")
files = os.listdir(in_cohort_dir)
print(files)

# 2. Find appropriate files with more flexible pattern matching
soft_file = None
matrix_file = None

for file in files:
    file_path = os.path.join(in_cohort_dir, file)
    # Look for files that might contain SOFT or matrix data with various possible extensions
    if 'soft' in file.lower() or 'family' in file.lower() or file.endswith('.soft.gz'):
        soft_file = file_path
    if 'matrix' in file.lower() or file.endswith('.txt.gz') or file.endswith('.tsv.gz'):
        matrix_file = file_path

if not soft_file:
    print("Warning: Could not find a SOFT file. Using the first .gz file as fallback.")
    gz_files = [f for f in files if f.endswith('.gz')]
    if gz_files:
        soft_file = os.path.join(in_cohort_dir, gz_files[0])

if not matrix_file:
    print("Warning: Could not find a matrix file. Using the second .gz file as fallback if available.")
    gz_files = [f for f in files if f.endswith('.gz')]
    if len(gz_files) > 1 and soft_file != os.path.join(in_cohort_dir, gz_files[1]):
        matrix_file = os.path.join(in_cohort_dir, gz_files[1])
    elif len(gz_files) == 1 and not soft_file:
        matrix_file = os.path.join(in_cohort_dir, gz_files[0])

print(f"SOFT file: {soft_file}")
print(f"Matrix file: {matrix_file}")

# 3. Read files if found
if soft_file and matrix_file:
    # Read the matrix file to obtain background information and sample characteristics data
    background_prefixes = ['!Series_title', '!Series_summary', '!Series_overall_design']
    clinical_prefixes = ['!Sample_geo_accession', '!Sample_characteristics_ch1']
    
    try:
        background_info, clinical_data = get_background_and_clinical_data(matrix_file, background_prefixes, clinical_prefixes)
        
        # Obtain the sample characteristics dictionary from the clinical dataframe
        sample_characteristics_dict = get_unique_values_by_row(clinical_data)
        
        # Explicitly print out all the background information and the sample characteristics dictionary
        print("Background Information:")
        print(background_info)
        print("Sample Characteristics Dictionary:")
        print(sample_characteristics_dict)
    except Exception as e:
        print(f"Error processing files: {e}")
        # Try swapping files if first attempt fails
        print("Trying to swap SOFT and matrix files...")
        temp = soft_file
        soft_file = matrix_file
        matrix_file = temp
        try:
            background_info, clinical_data = get_background_and_clinical_data(matrix_file, background_prefixes, clinical_prefixes)
            sample_characteristics_dict = get_unique_values_by_row(clinical_data)
            print("Background Information:")
            print(background_info)
            print("Sample Characteristics Dictionary:")
            print(sample_characteristics_dict)
        except Exception as e:
            print(f"Still error after swapping: {e}")
else:
    print("Could not find necessary files for processing.")


Files in the directory:
['GSE89231_family.soft.gz', 'GSE89231_series_matrix.txt.gz']
SOFT file: ../../input/GEO/Metabolic_Rate/GSE89231/GSE89231_family.soft.gz
Matrix file: ../../input/GEO/Metabolic_Rate/GSE89231/GSE89231_series_matrix.txt.gz
Background Information:
!Series_title	"Doxorubicin response in diffuse large B-cell lymphoma cell lines varies with concentration, exposure duration, and level of intrinsic sensitivity"
!Series_summary	"Although diffuse large B-cell lymphoma (DLBCL) is a very heterogeneous disease, patients are as standard treated with a combination of rituximab, cyclophosphamide, doxorubicin, vincristine, and prednisolone (R-CHOP). Since approximately 40% of patients die due to refractory disease or relapse, enhanced knowledge about drug response mechanisms is required to improve treatment outcome. Therefore, this study assesses parameters that possibly influence doxorubicin response. Doxorubicin-induced impact on the number of living cells was evaluated for four

### Step 2: Dataset Analysis and Clinical Feature Extraction

In [3]:
# 1. Analyze if the dataset contains gene expression data
# Based on the background information, this dataset includes gene expression data from DLBCL cell lines
is_gene_available = True

# 2.1 Data Availability Analysis
# Trait - Metabolic Rate - Not directly available in this dataset
# This dataset appears to be about doxorubicin sensitivity in DLBCL cell lines, which doesn't directly measure metabolic rate
trait_row = None  # No direct measurement of metabolic rate

# Age - Not applicable for cell lines
age_row = None

# Gender - Not applicable for cell lines
gender_row = None

# 2.2 Data Type Conversion Functions (though not used in this dataset)
def convert_trait(value):
    """Convert trait values to appropriate data type."""
    if value is None or pd.isna(value):
        return None
    if ':' in str(value):
        value = str(value).split(':', 1)[1].strip()
    # Additional conversion logic would go here if trait data were available
    return value

def convert_age(value):
    """Convert age values to continuous data type."""
    if value is None or pd.isna(value):
        return None
    if ':' in str(value):
        value = str(value).split(':', 1)[1].strip()
    # Additional conversion logic would go here if age data were available
    return value

def convert_gender(value):
    """Convert gender values to binary data type (0 for female, 1 for male)."""
    if value is None or pd.isna(value):
        return None
    if ':' in str(value):
        value = str(value).split(':', 1)[1].strip().lower()
    # Additional conversion logic would go here if gender data were available
    return value

# 3. Save Metadata - Initial Filtering
# Trait data is not available (trait_row is None)
is_trait_available = trait_row is not None
validate_and_save_cohort_info(
    is_final=False,
    cohort=cohort,
    info_path=json_path,
    is_gene_available=is_gene_available,
    is_trait_available=is_trait_available
)

# 4. Clinical Feature Extraction
# Since trait_row is None, we skip the clinical feature extraction step


False

### Step 3: Gene Data Extraction

In [4]:
# 1. First get the path to the soft and matrix files
soft_file, matrix_file = geo_get_relevant_filepaths(in_cohort_dir)

# 2. Looking more carefully at the background information
# This is a SuperSeries which doesn't contain direct gene expression data
# Need to investigate the soft file to find the subseries
print("This appears to be a SuperSeries. Looking at the SOFT file to find potential subseries:")

# Open the SOFT file to try to identify subseries
with gzip.open(soft_file, 'rt') as f:
    subseries_lines = []
    for i, line in enumerate(f):
        if 'Series_relation' in line and 'SuperSeries of' in line:
            subseries_lines.append(line.strip())
        if i > 1000:  # Limit search to first 1000 lines
            break

# Display the subseries found
if subseries_lines:
    print("Found potential subseries references:")
    for line in subseries_lines:
        print(line)
else:
    print("No subseries references found in the first 1000 lines of the SOFT file.")

# Despite trying to extract gene data, we expect it might fail because this is a SuperSeries
try:
    gene_data = get_genetic_data(matrix_file)
    print("\nGene data extraction result:")
    print("Number of rows:", len(gene_data))
    print("First 20 gene/probe identifiers:")
    print(gene_data.index[:20])
except Exception as e:
    print(f"Error extracting gene data: {e}")
    print("This confirms the dataset is a SuperSeries without direct gene expression data.")


This appears to be a SuperSeries. Looking at the SOFT file to find potential subseries:


No subseries references found in the first 1000 lines of the SOFT file.



Gene data extraction result:
Number of rows: 54675
First 20 gene/probe identifiers:
Index(['1007_s_at', '1053_at', '117_at', '121_at', '1255_g_at', '1294_at',
       '1316_at', '1320_at', '1405_i_at', '1431_at', '1438_at', '1487_at',
       '1494_f_at', '1552256_a_at', '1552257_a_at', '1552258_at', '1552261_at',
       '1552263_at', '1552264_a_at', '1552266_at'],
      dtype='object', name='ID')


### Step 4: Gene Identifier Review

In [5]:
# Looking at the gene identifiers, they appear to be Affymetrix probe IDs (e.g., '1007_s_at', '1053_at'),
# not standard human gene symbols. These will need to be mapped to proper gene symbols.

requires_gene_mapping = True


### Step 5: Gene Annotation

In [6]:
# 1. Use the 'get_gene_annotation' function from the library to get gene annotation data from the SOFT file.
gene_annotation = get_gene_annotation(soft_file)

# 2. Use the 'preview_df' function from the library to preview the data and print out the results.
print("Gene annotation preview:")
print(preview_df(gene_annotation))


Gene annotation preview:
{'ID': ['1007_s_at', '1053_at', '117_at', '121_at', '1255_g_at'], 'GB_ACC': ['U48705', 'M87338', 'X51757', 'X69699', 'L36861'], 'SPOT_ID': [nan, nan, nan, nan, nan], 'Species Scientific Name': ['Homo sapiens', 'Homo sapiens', 'Homo sapiens', 'Homo sapiens', 'Homo sapiens'], 'Annotation Date': ['Oct 6, 2014', 'Oct 6, 2014', 'Oct 6, 2014', 'Oct 6, 2014', 'Oct 6, 2014'], 'Sequence Type': ['Exemplar sequence', 'Exemplar sequence', 'Exemplar sequence', 'Exemplar sequence', 'Exemplar sequence'], 'Sequence Source': ['Affymetrix Proprietary Database', 'GenBank', 'Affymetrix Proprietary Database', 'GenBank', 'Affymetrix Proprietary Database'], 'Target Description': ['U48705 /FEATURE=mRNA /DEFINITION=HSU48705 Human receptor tyrosine kinase DDR gene, complete cds', 'M87338 /FEATURE= /DEFINITION=HUMA1SBU Human replication factor C, 40-kDa subunit (A1) mRNA, complete cds', "X51757 /FEATURE=cds /DEFINITION=HSP70B Human heat-shock protein HSP70B' gene", 'X69699 /FEATURE= /DEF

### Step 6: Gene Identifier Mapping

In [7]:
# 1. Identify the appropriate columns for gene identifiers and gene symbols
# From the previews, we can see:
# - Gene expression data uses identifiers like '1007_s_at' which are in the 'ID' column of the gene annotation
# - Gene symbols are stored in the 'Gene Symbol' column of the gene annotation

# 2. Get a gene mapping dataframe by extracting the ID and Gene Symbol columns
mapping_df = get_gene_mapping(gene_annotation, prob_col='ID', gene_col='Gene Symbol')

# Print a preview of the mapping dataframe
print("Gene mapping preview:")
print(mapping_df.head())

# 3. Apply the gene mapping to convert probe-level measurements to gene expression data
gene_data = apply_gene_mapping(gene_data, mapping_df)

# Print information about the resulting gene expression data
print("\nGene expression data after mapping:")
print(f"Number of genes: {len(gene_data)}")
print(f"Number of samples: {gene_data.shape[1]}")
print("First few gene symbols:")
print(gene_data.index[:10])

# Normalize gene symbols to handle synonyms and duplicates
gene_data = normalize_gene_symbols_in_index(gene_data)
print("\nAfter normalizing gene symbols:")
print(f"Number of genes after normalization: {len(gene_data)}")
print("First few normalized gene symbols:")
print(gene_data.index[:10])


Gene mapping preview:
          ID              Gene
0  1007_s_at  DDR1 /// MIR4640
1    1053_at              RFC2
2     117_at             HSPA6
3     121_at              PAX8
4  1255_g_at            GUCA1A



Gene expression data after mapping:
Number of genes: 21278
Number of samples: 49
First few gene symbols:
Index(['A1BG', 'A1BG-AS1', 'A1CF', 'A2M', 'A2M-AS1', 'A2ML1', 'A2MP1',
       'A4GALT', 'A4GNT', 'AA06'],
      dtype='object', name='Gene')

After normalizing gene symbols:
Number of genes after normalization: 19845
First few normalized gene symbols:
Index(['A1BG', 'A1BG-AS1', 'A1CF', 'A2M', 'A2M-AS1', 'A2ML1', 'A2MP1',
       'A4GALT', 'A4GNT', 'AA06'],
      dtype='object', name='Gene')


### Step 7: Data Normalization and Linking

In [8]:
# 1. Normalize gene symbols in the gene expression data
gene_data_normalized = normalize_gene_symbols_in_index(gene_data)
print(f"Shape of gene data after normalization: {gene_data_normalized.shape}")
os.makedirs(os.path.dirname(out_gene_data_file), exist_ok=True)
gene_data_normalized.to_csv(out_gene_data_file)
print(f"Saved normalized gene data to {out_gene_data_file}")

# 2. Acknowledge that trait data is not available
print("No metabolic rate data is available in this dataset.")

# Extract sample IDs for potential clinical data
sample_ids = gene_data.columns.tolist()
print(f"Number of samples: {len(sample_ids)}")

# Create a minimal clinical dataframe with only sample IDs
clinical_df = pd.DataFrame(index=sample_ids)

# Save the clinical data (contains only sample IDs)
os.makedirs(os.path.dirname(out_clinical_data_file), exist_ok=True)
clinical_df.to_csv(out_clinical_data_file)
print(f"Saved clinical data to {out_clinical_data_file}")

# 3. Skip linking since trait data is not available
print("Skipping data linking as trait data (Metabolic_Rate) is not available in this dataset.")

# 4. Validate the dataset as unusable due to missing trait data
# Use is_final=False since we don't have valid trait data to evaluate bias
validate_and_save_cohort_info(
    is_final=False,
    cohort=cohort,
    info_path=json_path,
    is_gene_available=True,
    is_trait_available=False  # No trait data available
)

# 5. Since the dataset is not usable for our trait, we don't save the linked data
print("Dataset is not usable for Metabolic_Rate analysis. No linked data saved.")

Shape of gene data after normalization: (19845, 49)


Saved normalized gene data to ../../output/preprocess/Metabolic_Rate/gene_data/GSE89231.csv
No metabolic rate data is available in this dataset.
Number of samples: 49
Saved clinical data to ../../output/preprocess/Metabolic_Rate/clinical_data/GSE89231.csv
Skipping data linking as trait data (Metabolic_Rate) is not available in this dataset.
Dataset is not usable for Metabolic_Rate analysis. No linked data saved.
